In [1]:
import torch
from einops import rearrange

# Example tensor with shape (b=1, c=1, h=2, w=2)
input_tensor = torch.arange(1, (12*8)+1).reshape(1, 1, 12, 8)
patch_height = 3
patch_width = 4

output_tensor = rearrange(input_tensor, 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=patch_height, p2=patch_width)


/home2/ebrahim/miniconda3/envs/llm_brain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home2/ebrahim/miniconda3/envs/llm_brain/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [19]:
input_tensor

tensor([[[[ 1,  2,  3,  4,  5,  6,  7,  8],
          [ 9, 10, 11, 12, 13, 14, 15, 16],
          [17, 18, 19, 20, 21, 22, 23, 24],
          [25, 26, 27, 28, 29, 30, 31, 32],
          [33, 34, 35, 36, 37, 38, 39, 40],
          [41, 42, 43, 44, 45, 46, 47, 48],
          [49, 50, 51, 52, 53, 54, 55, 56],
          [57, 58, 59, 60, 61, 62, 63, 64],
          [65, 66, 67, 68, 69, 70, 71, 72],
          [73, 74, 75, 76, 77, 78, 79, 80],
          [81, 82, 83, 84, 85, 86, 87, 88],
          [89, 90, 91, 92, 93, 94, 95, 96]]]])

In [20]:
output_tensor

tensor([[[ 1,  2,  3,  4,  9, 10, 11, 12, 17, 18, 19, 20],
         [ 5,  6,  7,  8, 13, 14, 15, 16, 21, 22, 23, 24],
         [25, 26, 27, 28, 33, 34, 35, 36, 41, 42, 43, 44],
         [29, 30, 31, 32, 37, 38, 39, 40, 45, 46, 47, 48],
         [49, 50, 51, 52, 57, 58, 59, 60, 65, 66, 67, 68],
         [53, 54, 55, 56, 61, 62, 63, 64, 69, 70, 71, 72],
         [73, 74, 75, 76, 81, 82, 83, 84, 89, 90, 91, 92],
         [77, 78, 79, 80, 85, 86, 87, 88, 93, 94, 95, 96]]])

What einops rearange does is it changes the input so to be of shape num patches.    
Each patch is of shape patch width * patch height. Patches are arranged so that
patches nearby in time are next to each other.      
So every block of T/patch_height patches
should be able to attend to each other. 

In [31]:
def make_blockwise_mask(num_patches, patch_width, num_features, device='cpu'):
    """
    Creates a (num_patches, num_patches) boolean mask where each non-overlapping group of N patches
    can attend only to each other.

    Args:
        num_patches (int): total number of tokens (patches)
        N (int): number of patches per block (must divide num_patches evenly)
        device (str or torch.device): where to create the mask

    Returns:
        mask (torch.Tensor): boolean tensor of shape (num_patches, num_patches)
                             with True where attention is allowed
    """
    
    N = num_features / patch_width
    assert num_patches % N == 0, "num_patches must be divisible by N"
    
    block_id = torch.arange(num_patches, device=device) // N  # Shape: (num_patches,)
    mask = block_id.unsqueeze(0) == block_id.unsqueeze(1)     # Shape: (num_patches, num_patches)
    
    return mask  # dtype: bool

mask = make_blockwise_mask(num_patches=9, num_features=6, patch_width=2)

print(mask.int())

tensor([[1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 1, 1]], dtype=torch.int32)
